## 0) 준비: import + 초기화 유틸

목적

- ViT는 보통 가중치 초기화로 truncated normal(std=0.02) 관례를 씁니다.

- (엄밀 버전은 timm 라이브러리가 더 낫지만, 정리용으로 간단 구현)

$$
erf(x) = \frac{2}{\sqrt{\pi}} \int_{0}^{x} e^{-t^2} dt
$$

$$GELU(x) = x \cdot \Phi(x) = 0.5x \left(1 + erf\left(\frac{x}{\sqrt{2}}\right)\right)$$

In [2]:
import math
import torch
import torch.nn as nn

def trunc_nomal_(tensor, mean=0.0, std=1.0, a=-2.0, b=2.0):
    # timm 스타일의 truncated normal 간단 구현
    # (엄밀한 구현이 필요하면 timm.trunc_normal_ 사용 권장)

    with torch.no_grad():
        l = 0.5 * (1.0 + math.erf((a - mean) / (std * math.sqrt(2.0)))) 
        u = 0.5 * (1.0 + math.erf((b - mean) / (std * math.sqrt(2.0))))
        tensor.uniform_(2 * l - 1, 2 * u - 1)
        tensor.erfinv_()
        tensor.mul_(std * math.sqrt(2.0)).add_(mean)
        tensor.clamp_(min=a, max=b)
        return tensor

## 1) Patch Embedding: Patchify + Linear Projection

**논문 단계**

1. 이미지를 P×P 크기로 쪼개서 patch들을 만든다 (patchify)
2. 각 patch를 펼쳐서 (P×P×C)
3. Linear로 D차원 토큰으로 바꾼다

**구현 포인트**

- 실제 구현은 `Conv2d(kernel=P, stride=P)`가 위 3단계를 한 번에 수행합니다.
- 출력은 Transformer가 먹기 좋은 **(B, N, D)** 토큰 시퀀스입니다.

shape 흐름:

- 입력: (B, C, H, W)
- conv: (B, D, H/P, W/P)
- flatten: (B, N, D) where N=(H/P)(W/P)

In [ ]:
class PatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = (img_size, img_size) if isinstance(img_size, int) else img_size
        patch_size = (patch_size, patch_size) if isinstance(patch_size, int) else patch_size

        self.img_size = img_size
        self.patch_size = patch_size

        assert img_size[0] % patch_size[0] == 0 and img_size[1] % patch_size[1] == 0, \
            "img_size must be divisible by patch_size"

        self.grid_size = (img_size[0] // patch_size[0], img_size[1] // patch_size[1])
        self.num_patches = self.grid_size[0] * self.grid_size[1]

        # Conv2d로 patchify + linear projection 동시 수행
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, C, H, W)
        x = self.proj(x) # (B, D, H/P, W/P), N = H/P * W/P
        x = x.flatten(start_dim=2).transpose(1, 2) # (B, N, D) where N = num_patches, D = embed_dim
        return x

In [13]:
torch.randn((2, 3, 4, 4)).flatten(2).shape

torch.Size([2, 3, 16])

## 2) MLP(Feed Forward): Linear → GELU → Linear

**논문 단계**

- Transformer block의 FFN/MLP 부분
- hidden_dim = mlp_ratio * D (보통 4D)
- GELU 활성화